# 1 ETL functions

* ETL 是 Extract, Transform, Load steps 這三個字的縮寫。所以其實就是我們熟悉的 data wrangling，只是它是用 CS 會用的詞彙而已
* ETL 具有以下兩個特性，所以很適合把他抓出來寫個 function，方便以後做自動化：
  * code 通常都很長(例如我們在 jumpstart 那章，就做了很多 data wrangling的事，最後才弄出一個乾淨漂亮的表格)
  * 久久才會改一次 code (因為 databases 通常不會常常在改變)  
* 那，我們這邊，就是要來寫一個 ETL function
  * 我把它命名為 `collect_data`，裡面的參數是放連線資訊
  * 然後我希望執行後，可以自動幫我連進資料庫、join三張table、做完資料清理，最後吐給我乾淨的整併後資料表
* 最後，補充一個很好用的 VSCode extension，叫 `Python Docstring Generator`，只要在定義function的時候，key入  `"""` 按下 enter，他就會自動幫你帶入 docstring 的模板，超好用啦！

In [ ]:
import sqlalchemy as sql
import pandas as pd

def collect_data(conn_string = "sqlite:///00_database/bike_orders_database.sqlite"):  
    """
    Collects and combines the bike orders data. 

    Args:
        conn_string (str, optional): A SQLAlchemy connection string to find the database. Defaults to "sqlite:///00_database/bike_orders_database.sqlite".

    Returns:
        DataFrame: A pandas data frame that combines data from tables:
            - orderlines: Transactions data
            - bikes: Products data
            - bikeshops: Customers data
    """

    # 1.0 Connect to database

    engine = sql.create_engine(conn_string)

    conn = engine.connect()

    table_names = ['bikes', 'bikeshops', 'orderlines']

    data_dict = {}
    for table in table_names:
        data_dict[table] = pd.read_sql(f"SELECT * FROM {table}", con=conn) \
            .drop("index", axis=1)
    
    conn.close()

    # 2.0 Combining Data

    df = pd.DataFrame(data_dict['orderlines']) \
        .merge(
            right    = data_dict['bikes'],
            how      = 'left',
            left_on  = 'product.id',
            right_on = 'bike.id'
        ) \
        .merge(
            right    = data_dict['bikeshops'],
            how      = "left",
            left_on  = "customer.id",
            right_on = 'bikeshop.id'
        )

    # 3.0 Cleaning Data 

    df['order.date'] = pd.to_datetime(df['order.date'])

    temp_df = df['description'].str.split(" - ", expand = True)
    df['category.1'] = temp_df[0]
    df['category.2'] = temp_df[1]
    df['frame.material'] = temp_df[2]

    temp_df = df['location'].str.split(", ", expand = True)
    df['city'] = temp_df[0]
    df['state'] = temp_df[1]

    df['total.price'] = df['quantity'] * df['price']

    cols_to_keep_list = [
        'order.id', 'order.line', 'order.date',    
        'quantity', 'price', 'total.price', 
        'model', 'category.1', 'category.2', 'frame.material', 
        'bikeshop.name', 'city', 'state'
    ]

    df = df[cols_to_keep_list]

    df.columns = df.columns.str.replace(".", "_")

    return df

* 來試試看效果吧：

In [ ]:
collect_data()

# 2 Package and Module

* 剛剛寫完 `collect_data` 這個 function 後，就變得很好用。  
* 但我們不想每次都要把這個 funtion 在文件的前面先定義一次，然後才開始用這個 funtion。我們想像 R 那樣，寫一個 `.R` 檔，然後去 source 他
* 那在 python，他有他的名詞，去定義這些行為：  
  * Module: module 就是一個 `.py` 檔，裡面放著我定義好的 `function` 和 `class` (class現在還沒介紹到)，可以拿來供未來的 code 使用。所以，我可以把我的 `collect_data` 這個 function，以及這個 function 要 import 的 library (`pandas`, `sqlalchemy`)，都包在一個 `.py` 檔裡，叫他 `database.py`，那 `database.py` 就是一個 module。
  * Package: package 就是裝滿 module 的一個資料夾。所以，我可以新增一個資料夾，叫做 `my_pandas_extensions`，裡面就放 `database.py`。此時，`my_pandas_extensions` 這個資料夾，就被稱為 package
* 這邊還有最後一個小提醒，就是`my_pandas_extensions`這個資料夾，實際上還要再多放 `__init__.py` 這個檔案，python才會知道這個資料夾的角色是 package。那 `__init__.py` 的內容現在可以先留白，之後再說。
* 所以，在這個階段，我們就是
  * 新增完 `my_pandas_extensions` 這個資料夾
  * 新增完 `__init__.py` 這個檔案 (內容是空白的)
  * 新增完 `database.py` 這個檔案 (內容是 import pandas, sqlalchemy這兩個 package，並定義好 `collect_data` 這個 function)


# 3. 引用自己寫好的 package

In [1]:
import pandas as pd
from my_pandas_extensions.database import collect_data

collect_data()

,order_id,order_line,order_date,quantity,price,total_price,model,category_1,category_2,frame_material,bikeshop_name,city,state
0,1,1,2011-01-07,1,6070,6070,Jekyll Carbon 2,Mountain,Over Mountain,Carbon,Ithaca Mountain Climbers,Ithaca,NY
1,1,2,2011-01-07,1,5970,5970,Trigger Carbon 2,Mountain,Over Mountain,Carbon,Ithaca Mountain Climbers,Ithaca,NY
2,2,1,2011-01-10,1,2770,2770,Beast of the East 1,Mountain,Trail,Aluminum,Kansas City 29ers,Kansas City,KS
3,2,2,2011-01-10,1,5970,5970,Trigger Carbon 2,Mountain,Over Mountain,Carbon,Kansas City 29ers,Kansas City,KS
4,3,1,2011-01-10,1,10660,10660,Supersix Evo Hi-Mod Team,Road,Elite Road,Carbon,Louisville Race Equipment,Louisville,KY
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15639,2000,4,2015-12-25,1,2660,2660,CAAD Disc Ultegra,Road,Elite Road,Aluminum,Austin Cruisers,Austin,TX
15640,2000,5,2015-12-25,1,1350,1350,Trail 2,Mountain,Sport,Aluminum,Austin Cruisers,Austin,TX
15641,2000,6,2015-12-25,1,1680,1680,CAAD12 105,Road,Elite Road,Aluminum,Austin Cruisers,Austin,TX
15642,2000,7,2015-12-25,1,2880,2880,F-Si Carbon 4,Mountain,Cross Country Race,Carbon,Austin Cruisers,Austin,TX
